solving Newsvendor problem modelled as 2 stage stochastic program

let D be RV of demand --> Normal

our decision is to decide the number of products to be kept in inventory at the begining of each day
cost of overstocking is $c_o$ and cost of understocking is $c_u$

min $\displaystyle\sum_w c_oo_w + c_uu_w$

$ i - o_w + u_w = d_w  \quad \forall w \in \Omega$ 

In [1]:
import numpy as np
from pyomo.environ import *
import scipy.stats as st

import plotly.graph_objects as go

In [95]:
mu = 50
sig = 10

demands = np.random.normal(mu,sig,9000)
c_o = 120
c_u = 250

In [96]:
### scenarios 

fig = go.Figure()

fig.add_trace(go.Histogram(x=demands))

fig.show()

In [97]:
model = ConcreteModel()

model.inv = Var(domain=NonNegativeReals)

model.o = Var(range(len(demands)),domain=NonNegativeReals)
model.u = Var(range(len(demands)),domain=NonNegativeReals)

model.constraints = ConstraintList()

for i,d in enumerate(demands):
    model.constraints.add(expr = model.inv - model.o[i] + model.u[i] == d)

model.obj = Objective(expr = sum([c_o*model.o[i]+c_u*model.u[i] for i in range(len(demands))]))

model.pprint()

3 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any : 9000 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 

In [98]:
minotaur = SolverFactory("mbnb", executable='../CHL_Grouping/mbin/mbnb')
res = minotaur.solve(model, tee=False)
print(res.write())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 18001
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: 
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 1.2791316509246826
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
None


In [99]:
model.inv()

54.42855798865668

In [100]:
st.norm.ppf(c_u/(c_u+c_o),loc = mu,scale = sig)

54.556403137378794

Dual of NewsVendor (2SSP)

<center>

$\displaystyle\max_{\pi_\omega} \quad \pi_\omega^T (ex-d_\omega) \\$
$ s.t. \hspace{150pt} \\$
$c_u \geq \pi_\omega \geq -c_o$

</center>

In [101]:
def getDual(x,demand,co,cu):
    dual = []
    for d in demand:
        if d-x > 0:
            dual.append(cu)
        elif d-x < 0:
            dual.append(-co)
        else:
            dual.append(0)

    return np.array(dual) 

In [102]:
def getValue(dual,x,demand):
    return np.matmul(dual.T,(np.array(demand))-np.ones(len(demands))*x)

In [110]:
def getCost(x,demand,co,cu):
    sum = 0
    for d in demand:
        sum += max(0,x-d)*co + max(0,d-x)*cu
    
    return sum

In [111]:
vals = []
cost = []
m = []
c = []

rng = (40,70)
for x in range(rng[0],rng[1]):
    dual = getDual(x,demands,c_o,c_u)
    vals.append(getValue(dual,x,demands))
    cost.append(getCost(x,demands,c_o,c_u))
    m.append(sum(dual))
    c.append(np.matmul(dual.T,np.array(demands)))

fig = go.Figure()

fig.update_layout(
    height = 800,
    width = 800
)

fig.add_trace(go.Scatter(x=np.arange(rng[0],rng[1]), y=vals, name = 'sub Problem costs'))
fig.add_trace(go.Scatter(x=np.arange(rng[0],rng[1]), y=cost, name = 'scenario costs'))

for i in range(len(m)):
    fig.add_trace(go.Scatter(x=np.arange(rng[0],rng[1]), y=[c[i]-m[i]*x for x in np.arange(rng[0],rng[1])], opacity=0.2 ) )

fig.show()

In [104]:
vals

[25053182.503565174,
 23376415.00596673,
 21793735.177844822,
 20312600.970527574,
 18940483.678703208,
 17687297.81958428,
 16557003.931175284,
 15546736.82919763,
 14653653.106118413,
 13880956.217409948,
 13236762.760781046,
 12732604.322937647,
 12363944.923985021,
 12126507.550217805,
 12017698.662622733,
 12025281.033400303,
 12142145.775624575,
 12375863.007278124,
 12714170.422065115,
 13147407.04243128]

Modelling 2 stage SP for train timetable Allowance

let $\\$
$r_{i,i+1}$ be the ideal time required to traverse block section between stations $i$ and $i+1$ $\\$
$R_{i,i+1}$ be the Random Variable of traversal time for block $i$ - $i+1$ $\\$

$D_i$ be announced departure time of train at station $i$ $\\$
$a_i \ \& \  d_i$ be actual arrivals and departures at station $i$ $\\$

Model : 

<center>

$\min \displaystyle\sum_{\omega \in \Omega} p_\omega \left( \sum_i^n E_ie_i^\omega + L_il_i^\omega  \right)$

</center>

$\qquad \hspace{250pt} $ s.t. 

<center>

$D_i \geq D_{i-1} + r_{i-1,i} + h_i \hspace{20pt} \forall i \in S$

$\hspace{30pt} a_i^\omega = d_{i-1}^\omega + R_{i-1,i}^\omega \hspace{40pt} \forall i \in S, \omega \in \Omega$

$\hspace{30pt} a_i^\omega + h_i + e_i^\omega - l_i^\omega = D_i \hspace{20pt} \forall i \in S, \omega \in \Omega$

$\hspace{30pt} d_i^\omega = D_i + l_i^\omega \hspace{60pt} \forall i \in S, \omega \in \Omega$

$D_i,e_i^\omega,l_i^\omega \geq 0$

</center>